#### Dependency Setup
----

In [ ]:
# Import dependencies
import datetime as dt
import json
import pandas as pd
import psycopg2
import psycopg2.extras as extras
import requests

from bs4 import BeautifulSoup
from jsonpath_ng.ext import parse

import warnings
warnings.filterwarnings('ignore')

#### Helper Functions
----

In [ ]:
# Create funtion for grabbing json sections
def getJSONSection(json_data, section):
    return [obj for obj in json_data if obj['name'] == section]

In [ ]:
# Create function for grabbing json values
def getJSONValue(json_data, section, stat):
    section_data = getJSONSection(json_data['splits']['categories'], section)
    stat_data = getJSONSection(section_data[0]['stats'], stat)
    return stat_data[0]['displayValue']

In [ ]:
# Function used to convert the team name retrieved from statistics API to the team name used by the league.  Used for looking up player photos
def getTeamUrl(team_name):
    # laliga
    if (team_name == 'Real Sociedad'):
        return 'real-sociedad'
    if (team_name == 'Villarreal'):
        return 'villarreal-cf'
    if (team_name == 'Valencia'):
        return 'valencia-cf'
    if (team_name == 'Almería'):
        return 'ud-almeria'
    if (team_name == 'Mallorca'):
        return 'rcd-mallorca'
    if (team_name == 'Sevilla'):
        return 'sevilla-fc'
    if (team_name == 'Athletic Club'):
        return 'athletic-club'
    if (team_name == 'Rayo Vallecano'):
        return 'rayo-vallecano'
    if (team_name == 'Cádiz'):
        return 'cadiz-cf'
    if (team_name == 'Atletico Madrid'):
        return 'atletico-de-madrid'
    if (team_name == 'Granada'):
        return 'granada-cf'
    if (team_name == 'Real Madrid'):
        return 'real-madrid'
    if (team_name == 'Real Betis'):
        return 'real-betis'
    if (team_name == 'Celta Vigo'):
        return 'rc-celta'
    if (team_name == 'Getafe'):
        return 'getafe-cf'
    if (team_name == 'Osasuna'):
        return 'c-a-osasuna'
    if (team_name == 'Girona'):
        return 'girona-fc'
    if (team_name == 'Las Palmas'):
        return 'ud-las-palmas'
    if (team_name == 'Barcelona'):
        return 'fc-barcelona'
    if (team_name == 'Alavés'):
        return 'd-alaves'

    # bundesliga
    if (team_name == 'TSG Hoffenheim'):
          return 'tsg-hoffenheim'
    if (team_name == '1. FC Union Berlin'):
        return '1-fc-union-berlin'
    if (team_name == 'Borussia Dortmund'):
        return 'borussia-dortmund'
    if (team_name == 'Bayern Munich'):
        return 'fc-bayern-muenchen'
    if (team_name == 'FC Cologne'):
        return '1-fc-koeln'
    if (team_name == 'VfB Stuttgart'):
        return 'vfb-stuttgart'
    if (team_name == 'Eintracht Frankfurt'):
        return 'eintracht-frankfurt'
    if (team_name == '1. FC Heidenheim 1846'):
        return '1-fc-heidenheim-1846'
    if (team_name == 'RB Leipzig'):
        return 'rb-leipzig'
    if (team_name == 'Borussia Monchengladbach'):
        return 'borussia-moenchengladbach'
    if (team_name == 'SC Freiburg'):
        return 'sport-club-freiburg'
    if (team_name == 'Mainz'):
        return '1-fsv-mainz-05'
    if (team_name == 'Bayer Leverkusen'):
        return 'bayer-04-leverkusen'
    if (team_name == 'Werder Bremen'):
        return 'sv-werder-bremen'
    if (team_name == 'VfL Bochum'):
        return 'vfl-bochum-1848'
    if (team_name == 'VfL Wolfsburg'):
        return 'vfl-wolfsburg'
    if (team_name == 'FC Augsburg'):
        return 'fc-augsburg'
    if (team_name == 'SV Darmstadt 98'):
        return 'sv-darmstadt-98'

    # liga mx
    if (team_name == 'Pumas UNAM'):
        return "18/pumas"
    if (team_name == 'Monterrey'):
         return "14/monterrey"
    if (team_name == 'FC Juarez'):
         return "11790/fc-juarez"
    if (team_name == 'Pachuca'):
         return "11/pachuca"
    if (team_name == 'Querétaro'):
         return "12037/queretaro"
    if (team_name == 'Mazatlán FC'):
         return "12043/mazatlan-fc"
    if (team_name == 'Tigres UANL'):
        return '16/tigres'
    if (team_name == 'León'):
        return '9/leon'
    if (team_name == 'América'):
        return '1/america'
    if (team_name == 'Cruz Azul'):
        return '12566/cruz-azul'
    if (team_name == 'Atlético de San Luis'):
        return '11220/atletico-de-san-luis'
    if (team_name == 'Tijuana'):
        return '5/tijuana'
    if (team_name == 'Puebla'):
        return '11550/puebla'
    if (team_name == 'Guadalajara'):
        return '7/guadalajara'
    if (team_name == 'Toluca'):
        return '17/toluca'
    if (team_name == 'Atlas'):
        return '10445/atlas'
    if (team_name == 'Santos'):
        return '15/santos-laguna'
    if (team_name == 'Necaxa'):
        return '29/necaxa'
    

    # ligue 1
    if (team_name == 'Metz'):
          return "fc-metz"
    if (team_name == 'Nice'):
          return "ogc-nice"
    if (team_name == 'Paris Saint-Germain'):
         return "paris-saint-germain"
    if (team_name == 'Lille'):
        return "losc-lille"
    if (team_name == 'Lyon'):
          return "olympique-lyonnais"
    if (team_name == 'Stade Rennais'):
         return "stade-rennais-fc"
    if (team_name == 'Nantes'):
          return "fc-nantes"
    if (team_name == 'Montpellier'):
         return "montpellier-herault-sc"
    if (team_name == 'Strasbourg'):
        return 'rc-strasbourg-alsace'
    if (team_name == 'Clermont Foot'):
        return 'clermont-foot-63'
    if (team_name == 'Lens'):
        return 'rc-lens'
    if (team_name == 'Marseille'):
        return 'olympique-de-marseille'
    if (team_name == 'Le Havre AC'):
        return 'havre-ac'
    if (team_name == 'Brest'):
        return 'stade-brestois-29'
    if (team_name == 'Lorient'):
        return 'fc-lorient'
    if (team_name == 'AS Monaco'):
        return 'as-monaco'
    if (team_name == 'Stade de Reims'):
        return 'stade-de-reims'
    if (team_name == 'Toulouse'):
        return 'toulouse-fc'
    

    # serie A
    if (team_name == 'Hellas Verona'):
         return "VERONA"
    if (team_name == 'Genoa'):
         return "GENOA"
    if (team_name == 'Sassuolo'):
         return "SASSU"
    if (team_name == 'Bologna'):
         return "BOLO"
    if (team_name == 'Fiorentina'):
         return "FIORE"
    if (team_name == 'AS Roma'):
         return "ROMA"
    if (team_name == 'AC Milan'):
         return "MILAN"
    if (team_name == 'Atalanta'):
         return "ATALAN"
    if (team_name == 'Cagliari'):
         return "CAGLIA"
    if (team_name == 'Monza'):
        return 'MONZA'
    if (team_name == 'Internazionale'):
        return 'INTER'
    if (team_name == 'Udinese'):
        return 'UDINES'
    if (team_name == 'Salernitana'):
        return 'SALER'
    if (team_name == 'Lazio'):
        return 'LAZIO'
    if (team_name == 'Juventus'):
        return 'JUVE'
    if (team_name == 'Frosinone'):
        return 'FROSI'
    if (team_name == 'Empoli'):
        return 'EMPOLI'
    if (team_name == 'Torino'):
        return 'TORINO'
    if (team_name == 'Lecce'):
        return 'LECCE'
    if (team_name == 'Napoli'):
        return 'NAPOLI'
    

In [ ]:
# return the db cursor.  DB name is a parameter, since we need to use postgres to create the soccer_stats db, then need soccer_stats to create the tables and insert data.
def getDBCursor(dbName):
    
    conn = psycopg2.connect(
        database=f"{dbName}", user='postgres', password='postgres', host='127.0.0.1', port='5432'
    )

    conn.autocommit = True

    cursor = conn.cursor()
    
    return cursor

In [ ]:
# create the soccer_stats db in postgres
def createSoccerStatsDB():
    
    db_cursor = getDBCursor('postgres')

    db_cursor.execute("DROP DATABASE IF EXISTS soccer_stats WITH (FORCE)")
    db_cursor.execute("CREATE DATABASE soccer_stats")

    db_cursor.connection.close()

In [ ]:
# create the soccer_stats tables
def createSoccerStatsTables():
    
    db_cursor = getDBCursor('soccer_stats')

    db_cursor.execute(open("schema/create_tables.sql", "r").read())
    
    db_cursor.connection.close()

In [ ]:
# insert DataFrame data into the provided table
def writeDataFrameToDB(df, table):
    data_tuples = [tuple(data) for data in df.to_numpy()]
    columns = ','.join(list(df.columns))
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, columns) 
    cursor = getDBCursor('soccer_stats')
    
    extras.execute_values(cursor, query, data_tuples)
    cursor.connection.commit()
    

In [ ]:
# Returns weight in < nnn lbs > string format as integer
def getWeightAsInt(weight):
    try:
        weight_nbr = weight.split(' ')[0]
    except:
        weight_nbr = 0

    return weight_nbr if weight_nbr != 'N/A' else 0

In [ ]:
# Returns height in < n' n" > string format as inches as integer
def getHeightAsInteger(height):
    try:
        numbers = height.replace('\'', '').replace('"','').split(' ')
        feet = int(numbers[0])
        inches = int(numbers[1])
        total_inches = (feet * 12) + inches
        height_nbr = total_inches
    except:
        height_nbr = 0

    return height_nbr if height_nbr != 'N/A' else 0

In [ ]:
# Each league has a different way of obtaining a player photo.
# This function takes in the league name and executes the method appropriate for that league
def getPlayerPhotoURL(league_name, team_results_json, athlete_results_json):
    if league_name == 'Premier League':
        try:
            name_search_url = f"https://footballapi.pulselive.com/search/PremierLeague/?terms={athlete_results_json['displayName']}&type=player&size=1&start=0&fullObjectResponse=true"
            name_search_result_json = requests.get(name_search_url, headers = {"origin": "https://www.premierleague.com"}).json()

            image_id = name_search_result_json['hits']['hit'][0]['response']['altIds']['opta']
            image_url = f"https://resources.premierleague.com/premierleague/photos/players/250x250/{image_id}.png"
        except:
            image_url = 'N/A'

    elif league_name == 'LALIGA':
      try:
        club_squad_url = f"https://www.laliga.com/en-US/clubs/{getTeamUrl(team_results_json['displayName'])}/squad"
        club_squad_results = requests.get(club_squad_url)
        club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))
        club_squad_json = json.loads(club_squad_results_parsed.body.find('script', attrs={'id': '__NEXT_DATA__'}).text)
        player_element = [obj for obj in club_squad_json['props']['pageProps']['squad']['squads'] if ('shirt_number' in obj) and (str(obj['shirt_number'])) == athlete_results_json['jersey']]
        image_url =  player_element[0]['photos']['002']['512x512']
      except:
        image_url = 'N/A'

    elif league_name == 'Bundesliga':
      try:
        club_squad_url = f"https://www.bundesliga.com/en/bundesliga/clubs/{getTeamUrl(team_results_json['displayName'])}"
        club_squad_results = requests.get(club_squad_url)
        club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))
        club_squad_json = json.loads(club_squad_results_parsed.body.find('script', attrs={'id': 'serverApp-state'}).text)
        jsonpath_expr = parse(f"$..players.*[?(@.shirtNumber={athlete_results_json['jersey']})]")
        match = [items.value['playerImages']['FACE_CIRCLE'].replace('-circle', '') for items in jsonpath_expr.find(club_squad_json)]
        image_url = match[0]
      except:
        image_url = 'N/A'

    elif league_name == 'MLS':
      try:
        image_url = athlete_results_json['headshot']['href'] if 'headshot' in athlete_results_json else 'N/A'
      except:
        image_url = 'N/A'

    elif league_name == 'NWSL':
      try:
        base_url = 'https://d2nkt8hgeld8zj.cloudfront.net'
        player_list_url = f'{base_url}/services/nwsl.ashx/players'

        player_list_results = requests.get(player_list_url)
        player_list_results_results_json = player_list_results.json()
              
        jsonpath_expr = parse(f"$.data[?(@.shirtNumber=='{athlete_results_json['jersey']}' & @.team.title == '{team_results_json['displayName']}')]")
        match = [items.value['images']['head_shot']['url'] for items in jsonpath_expr.find(player_list_results_results_json)]
        image_url = f'{base_url}{match[0]}' if len(match) > 0 else 'N/A'
      except:
        image_url = 'N/A'

    elif league_name == 'Liga MX':
      try:
        club_squad_url = f"https://ligamx.net/cancha/club/{getTeamUrl(team_results_json['displayName'])}"
        club_squad_results = requests.get(club_squad_url)
        club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))

        match = [number.parent.parent.parent.find('img') for number in club_squad_results_parsed.find_all('strong', {'class': 'numero'}) if number.get_text() == f"#{athlete_results_json['jersey']}"]
        image_url = match[0]['src'] if len(match) > 0 else 'N/A'
      except:
        image_url = 'N/A'

    elif league_name == 'Ligue 1':
      try:
        base_url = "https://www.ligue1.com"
        club_squad_url = f"{base_url}/clubs/squad?id={getTeamUrl(team_results_json['displayName'])}"
        club_squad_results = requests.get(club_squad_url)
        club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))

        match = [number.parent.find('img') for number in club_squad_results_parsed.find_all('div', {'class': 'SquadTeamTable-detail--number'}) if number.get_text(strip=True) == athlete_results_json['jersey']]
        image_url = f"{base_url}{match[0]['src']}" if len(match) > 0 else 'N/A'
      except:
        image_url = 'N/A'

    elif league_name == 'Serie A':
      try:
        player_results = requests.get(f"https://www.legaseriea.it/api/team/{getTeamUrl(team_results_json['displayName'])}/players")
        player_results_json = player_results.json()

        jsonpath_expr = parse(f"$..data.*[?(@.uniform_number={athlete_results_json['jersey']})]")
        match = [items.value['medium_shot'] for items in jsonpath_expr.find(player_results_json)]
        image_url = match[0]
      except:
        image_url = 'N/A'

    return image_url


#### Data Setup
----

In [ ]:
# List of dictionaries that maps league name to API
soccer_leagues = [
    {
        'display': 'MLS',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'MLS',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.1/seasons/2023/athletes?lang=en&region=us&limit=1000&page=2'
    },
    {
        'display': 'NWSL',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.nwsl/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Premier League',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/eng.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'LALIGA',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/esp.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'LALIGA',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/esp.1/seasons/2023/athletes?lang=en&region=us&limit=1000&page=2'
    },
    {
        'display': 'Bundesliga',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/ger.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Bundesliga',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/ger.1/seasons/2023/athletes?lang=en&region=us&limit=1000&page=2'
    },
    {
        'display': 'Liga MX',
        'url': 'https://sports.core.api.espn.com/v2/sports/soccer/leagues/mex.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Ligue 1',
        'url': 'https://sports.core.api.espn.com/v2/sports/soccer/leagues/fra.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Serie A',
        'url': 'https://sports.core.api.espn.com/v2/sports/soccer/leagues/ita.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    }
]

In [ ]:
# Set the leagues to gather statistics 
selected_leagues = ['MLS','NWSL','Premier League','LALIGA','Bundesliga','Liga MX','Ligue 1','Serie A']

#### Data Collection
----

In [ ]:
# Get the data

all_player_data = []

# used to track retrieval status
record_counter = 0

for league in soccer_leagues:
    if (league['display'] in selected_leagues):
        athlete_list_results_json = requests.get(league['url']).json()

        #for athlete in range(len(athlete_list_results_json['items'])):
        for athlete in range(15):
            record_counter += 1
            print(record_counter)
            print(athlete_list_results_json['items'][athlete]['$ref'])
            athlete_results_json = requests.get(athlete_list_results_json['items'][athlete]['$ref']).json()

            # Skip this athlete if they don't have statistics or a team
            if ('statistics' in athlete_results_json and 'team' in athlete_results_json):
                
                #get the team 
                team_results_json = requests.get(athlete_results_json['team']['$ref']).json()

                # get the statistics
                athlete_stats_results_json = requests.get(athlete_results_json['statistics']['$ref']).json()
          
                # get the player's photo from the league's website
                player_photo_url = getPlayerPhotoURL(league['display'], team_results_json, athlete_results_json)

                player_data = {
                    "player_name" : athlete_results_json['displayName'],
                    "team" : team_results_json['displayName'],
                    "league" : league['display'],
                    "position" : athlete_results_json['position']['displayName'],
                    "jersey_number" : athlete_results_json['jersey']if 'jersey' in athlete_results_json else 'N/A',
                    "goals" : getJSONValue(athlete_stats_results_json, 'offensive', 'totalGoals'),
                    "assists" : getJSONValue(athlete_stats_results_json, 'offensive', 'goalAssists'),
                    "yellow_cards" : getJSONValue(athlete_stats_results_json, 'general', 'yellowCards'),
                    "red_cards" : getJSONValue(athlete_stats_results_json, 'general', 'redCards'),
                    "shots" : getJSONValue(athlete_stats_results_json, 'offensive', 'totalShots'),
                    "shots_on_goal" : getJSONValue(athlete_stats_results_json, 'offensive', 'shotsOnTarget'),
                    "saves" : getJSONValue(athlete_stats_results_json, 'goalKeeping', 'saves'),
                    "clean_sheets" : getJSONValue(athlete_stats_results_json, 'goalKeeping', 'cleanSheet'),
                    "goals_against" : getJSONValue(athlete_stats_results_json, 'goalKeeping', 'goalsConceded'),
                    "photo_url" : player_photo_url,
                    "height" : athlete_results_json['displayHeight'] if 'displayHeight' in athlete_results_json else 'N/A',
                    "weight" : athlete_results_json['displayWeight'] if 'displayWeight' in athlete_results_json else 'N/A',
                    "birthdate" : athlete_results_json['dateOfBirth']if 'dateOfBirth' in athlete_results_json else 'N/A',
                    "nationality" : athlete_results_json['citizenship']if 'citizenship' in athlete_results_json else 'N/A'
                }

                all_player_data.append(player_data)
            
        

#### Database Prep
----

In [ ]:
# create the PostgreSQL database and tables

createSoccerStatsDB()
createSoccerStatsTables()

#### Generate Relational Data
----

In [ ]:
# create dataframes for the SQL tables

athlete_stats_df = pd.DataFrame(all_player_data)

leagues_df = pd.DataFrame(athlete_stats_df.league.unique(), columns=['league'])
leagues_df.insert(0, 'league_id', range(1, 1 + len(leagues_df)))

positions_df = pd.DataFrame(athlete_stats_df.position.unique(), columns=['position'])
positions_df.insert(0, 'position_id', range(1, 1 + len(positions_df)))

nationalities_df = pd.DataFrame(athlete_stats_df.nationality.unique(), columns=['nationality'])
nationalities_df.insert(0, 'nationality_id', range(1, 1 + len(nationalities_df)))

team_starter_df = pd.DataFrame(athlete_stats_df.team.unique(), columns=["team"])
team_starter_df.insert(0, 'team_id', range(1, 1 + len(team_starter_df)))
team_athlete_df = team_starter_df.merge(athlete_stats_df)
team_athlete_league_df = team_athlete_df.merge(leagues_df)
team_df = team_athlete_league_df.drop_duplicates(subset=['team_id', 'league_id', 'team'])
team_df = team_df[['team_id', 'league_id', 'team']]

# merge dataframes to include ids
merged_df = athlete_stats_df.merge(leagues_df)
merged_df = merged_df.merge(positions_df)
merged_df = merged_df.merge(nationalities_df)
merged_df = merged_df.merge(team_df)


# clean up the data
merged_df['birthdate'] = merged_df['birthdate'].str.split('T', expand=True)[0]
merged_df['weight'] = [getWeightAsInt(weight) for weight in merged_df['weight']]
merged_df['height'] = [getHeightAsInteger(height) for height in merged_df['height']]
merged_df['jersey_number'] = [-1 if pd.isna(number) else number for number in merged_df['jersey_number']]
merged_df['photo_url'] = ['N/A' if pd.isnull(photo) else photo for photo in merged_df['photo_url']]
merged_df['birthdate'] = [dt.date.min if pd.isnull(birthdate) else birthdate for birthdate in merged_df['birthdate']]
merged_df = merged_df.astype({'jersey_number': 'int32'})

player_df = merged_df.drop(columns=['team', 'league', 'position', 'nationality', 'league_id'])
player_df.insert(0, 'player_id', range(1, 1 + len(player_df)))
player_df = player_df[[
    'player_id', 'player_name', 'team_id', 'position_id', 'jersey_number', 
    'goals', 'assists', 'yellow_cards', 'red_cards', 'shots', 'shots_on_goal', 
    'saves', 'clean_sheets', 'goals_against', 'photo_url', 'height', 
    'weight', 'birthdate', 'nationality_id'
]]


#### Load Data Into DB
----

In [ ]:
# add data to db tables

writeDataFrameToDB(leagues_df, 'league')
writeDataFrameToDB(nationalities_df, 'nationality')
writeDataFrameToDB(positions_df, 'position')
writeDataFrameToDB(team_df, 'team')
writeDataFrameToDB(player_df, 'player_data')
